### Import libraries and Dataset

In [2]:
# Import libraries
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
# Set the path
parent_dir = Path.cwd().parent
data_dir = parent_dir / 'data'
input_path = data_dir / 'Online Retail.xlsx'

# Load the dataset
df = pd.read_excel(input_path)

# Check the head
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


### Initial EDA and Data Cleaning

In [4]:
# Check the information about the dataset
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    541909 non-null  object        
 1   StockCode    541909 non-null  object        
 2   Description  540455 non-null  object        
 3   Quantity     541909 non-null  int64         
 4   InvoiceDate  541909 non-null  datetime64[ns]
 5   UnitPrice    541909 non-null  float64       
 6   CustomerID   406829 non-null  float64       
 7   Country      541909 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 33.1+ MB


It seems that some `CustomerID` values and `Description` are missing in the dataset.

In [5]:
# Calculate the ratio of missingness 
miss_ratio_customerid = df['CustomerID'].isna().sum() / len(df['Country']) * 100

miss_ratio_description = df['Description'].isna().sum() / len(df['Country']) * 100

print(f'Missing ration of CustomrID is {miss_ratio_customerid}%')
print(f'Missing ration of Description  is {miss_ratio_description}%')

Missing ration of CustomrID is 24.926694334288598%
Missing ration of Description  is 0.2683107311375157%


In [6]:
# Subset the rows with missing values and take a peek
df_missing = df[df['CustomerID'].isna() | df['Description'].isna()]

df_missing.head(10)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
622,536414,22139,NaN,56,2010-12-01 11:52:00,0.00,NaN,United Kingdom
1443,536544,21773,DECORATIVE ROSE BATHROOM BOTTLE,1,2010-12-01 14:32:00,2.51,NaN,United Kingdom
1444,536544,21774,DECORATIVE CATS BATHROOM BOTTLE,2,2010-12-01 14:32:00,2.51,NaN,United Kingdom
1445,536544,21786,POLKADOT RAIN HAT,4,2010-12-01 14:32:00,0.85,NaN,United Kingdom
1446,536544,21787,RAIN PONCHO RETROSPOT,2,2010-12-01 14:32:00,1.66,NaN,United Kingdom
1447,536544,21790,VINTAGE SNAP CARDS,9,2010-12-01 14:32:00,1.66,NaN,United Kingdom
1448,536544,21791,VINTAGE HEADS AND TAILS CARD GAME,2,2010-12-01 14:32:00,2.51,NaN,United Kingdom
1449,536544,21801,CHRISTMAS TREE DECORATION WITH BELL,10,2010-12-01 14:32:00,0.43,NaN,United Kingdom
1450,536544,21802,CHRISTMAS TREE HEART DECORATION,9,2010-12-01 14:32:00,0.43,NaN,United Kingdom
1451,536544,21803,CHRISTMAS TREE STAR DECORATION,11,2010-12-01 14:32:00,0.43,NaN,United Kingdom


In [7]:
# Check if CustomerID is missing whenever Description is missing
# Subset rows that have missing values in Description 
df_missing_desc = df[df['Description'].isna()]

df_missing_desc.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
622,536414,22139,NaN,56,2010-12-01 11:52:00,0.0,NaN,United Kingdom
1970,536545,21134,NaN,1,2010-12-01 14:32:00,0.0,NaN,United Kingdom
1971,536546,22145,NaN,1,2010-12-01 14:33:00,0.0,NaN,United Kingdom
1972,536547,37509,NaN,1,2010-12-01 14:33:00,0.0,NaN,United Kingdom
1987,536549,85226A,NaN,1,2010-12-01 14:34:00,0.0,NaN,United Kingdom


In [8]:
# Compare the number of total rows and the number of rows that has missing values in CustomerID
print(f'the number of rows in missing_desc is {len(df_missing_desc)}')
print(f"the number of rows that have missing values in CustomerID in missing desc is {df_missing_desc['CustomerID'].isna().sum()}")

the number of rows in missing_desc is 1454
the number of rows that have missing values in CustomerID in missing desc is 1454


### Insight of Missing Data
After examining the dataset, three key insights were identified regarding missing values:
1. **Whenever 'Description is missing, `CustomerID` is also missing.**
2. **The proportion of missing values in `Description` is approximately 0.27%.**
3. **The proportion of missing values in `CustomerID` is approximately 25%**

### Decisions on Handling Missing Values
1. **Drop rows with missing `Description`.**

   Since only about 0.27% of rows are affected and these rows also lack `CustomerID`, which means there removal has a negligible impact on the dataset and the records provide little meaningful information for analysis.

2. **Drop rows with missing `CustomerID`**

   Although around 25% of the records are affected, `CustomerID` is essential for performing customer segmentation - the main goal of this project. 
   
   On top of that, even after removing these rows, over **406,000 valid records** remain, which is more than sufficient for robust analysis.

In [9]:
# Make a copy of df to secure original dataset
df_copy_1 = df.copy()

# Drop the rows with missing values
df_drop = df_copy_1.dropna(subset=['Description', 'CustomerID'])

# Confirm whether missing values are dropped
df_drop.isna().sum()

InvoiceNo      0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
UnitPrice      0
CustomerID     0
Country        0
dtype: int64

In [10]:
# Check the information of the new dataset
df_drop.info()

<class 'pandas.core.frame.DataFrame'>
Index: 406829 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    406829 non-null  object        
 1   StockCode    406829 non-null  object        
 2   Description  406829 non-null  object        
 3   Quantity     406829 non-null  int64         
 4   InvoiceDate  406829 non-null  datetime64[ns]
 5   UnitPrice    406829 non-null  float64       
 6   CustomerID   406829 non-null  float64       
 7   Country      406829 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 27.9+ MB


Now, the missing values have been successfully dropped.

I will check `UnitPrice` and `Quantity` columns to ensure there are no negative values.

In [11]:
# Check the UnitPrice column
unit_nev = df_drop[df_drop['UnitPrice'] < 0]

# Check the Quantity column
quan_nev = df_drop[df_drop['Quantity']<0]

# Check whether they have negative values
print(f"The number of negative values in UnitPrice is {len(unit_nev['UnitPrice'])}")
print(f"The number of negative values in Quantity is {len(quan_nev['Quantity'])}")

The number of negative values in UnitPrice is 0
The number of negative values in Quantity is 8905


The Quantity columns seems to have lots of negative values. Let's investigate them.

In [12]:
# Check the head
quan_nev.head(10)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
141,C536379,D,Discount,-1,2010-12-01 09:41:00,27.50,14527.0,United Kingdom
154,C536383,35004C,SET OF 3 COLOURED FLYING DUCKS,-1,2010-12-01 09:49:00,4.65,15311.0,United Kingdom
235,C536391,22556,PLASTERS IN TIN CIRCUS PARADE,-12,2010-12-01 10:24:00,1.65,17548.0,United Kingdom
236,C536391,21984,PACK OF 12 PINK PAISLEY TISSUES,-24,2010-12-01 10:24:00,0.29,17548.0,United Kingdom
237,C536391,21983,PACK OF 12 BLUE PAISLEY TISSUES,-24,2010-12-01 10:24:00,0.29,17548.0,United Kingdom
238,C536391,21980,PACK OF 12 RED RETROSPOT TISSUES,-24,2010-12-01 10:24:00,0.29,17548.0,United Kingdom
239,C536391,21484,CHICK GREY HOT WATER BOTTLE,-12,2010-12-01 10:24:00,3.45,17548.0,United Kingdom
240,C536391,22557,PLASTERS IN TIN VINTAGE PAISLEY,-12,2010-12-01 10:24:00,1.65,17548.0,United Kingdom
241,C536391,22553,PLASTERS IN TIN SKULLS,-24,2010-12-01 10:24:00,1.65,17548.0,United Kingdom
939,C536506,22960,JAM MAKING SET WITH JARS,-6,2010-12-01 12:38:00,4.25,17897.0,United Kingdom


All invoice numbers start from `C`, which could indicate product cancellations. I will investigate this further.

At first, I hypothesized that for each cancellation(e.g. C536379), there might be corresponding original invoice without the `C` prefix(e.g.536379). To test this assumption, I will check whether such invoice numbers exist in the dataset.

In [13]:
# Get a list of invoice numbers from the from the first 10 rows
lst_inv_c = quan_nev.iloc[:10]['InvoiceNo'].to_list()
lst_inv_c 

['C536379',
 'C536383',
 'C536391',
 'C536391',
 'C536391',
 'C536391',
 'C536391',
 'C536391',
 'C536391',
 'C536506']

In [14]:
# Take out Cs from the invoice numbers and create a list
# Create an empty list to store the invoice numbers without C
lst_inv_nc = []

# Drop c from each element and append it to the list
for inv in lst_inv_c:
    lst_inv_nc.append(inv[1:])

lst_inv_nc

['536379',
 '536383',
 '536391',
 '536391',
 '536391',
 '536391',
 '536391',
 '536391',
 '536391',
 '536506']

In [15]:
# Merge the lists to subset the DataFrame
lst_inv_sub = lst_inv_c + lst_inv_nc
lst_inv_sub

['C536379',
 'C536383',
 'C536391',
 'C536391',
 'C536391',
 'C536391',
 'C536391',
 'C536391',
 'C536391',
 'C536506',
 '536379',
 '536383',
 '536391',
 '536391',
 '536391',
 '536391',
 '536391',
 '536391',
 '536391',
 '536506']

In [16]:
# Subset the rows
df_drop_inv_sub = df_drop[df_drop['InvoiceNo'].isin(lst_inv_sub )]

df_drop_inv_sub

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
141,C536379,D,Discount,-1,2010-12-01 09:41:00,27.50,14527.0,United Kingdom
154,C536383,35004C,SET OF 3 COLOURED FLYING DUCKS,-1,2010-12-01 09:49:00,4.65,15311.0,United Kingdom
235,C536391,22556,PLASTERS IN TIN CIRCUS PARADE,-12,2010-12-01 10:24:00,1.65,17548.0,United Kingdom
236,C536391,21984,PACK OF 12 PINK PAISLEY TISSUES,-24,2010-12-01 10:24:00,0.29,17548.0,United Kingdom
237,C536391,21983,PACK OF 12 BLUE PAISLEY TISSUES,-24,2010-12-01 10:24:00,0.29,17548.0,United Kingdom
238,C536391,21980,PACK OF 12 RED RETROSPOT TISSUES,-24,2010-12-01 10:24:00,0.29,17548.0,United Kingdom
239,C536391,21484,CHICK GREY HOT WATER BOTTLE,-12,2010-12-01 10:24:00,3.45,17548.0,United Kingdom
240,C536391,22557,PLASTERS IN TIN VINTAGE PAISLEY,-12,2010-12-01 10:24:00,1.65,17548.0,United Kingdom
241,C536391,22553,PLASTERS IN TIN SKULLS,-24,2010-12-01 10:24:00,1.65,17548.0,United Kingdom
939,C536506,22960,JAM MAKING SET WITH JARS,-6,2010-12-01 12:38:00,4.25,17897.0,United Kingdom


**Investigating Potential Original Transactions**

The corresponding invoice numbers without `C` prefix don't seem to exist. It seems that it sometimes represent **discounts, but not all the time**. Therefore, I will look for records that have exactly the same `StockCode` and `CustomerID` as **the last 10 Orders** with invoice numbers starting with `C`, to investigate whether they represent the original transactions.

I chose **the last 10 records** because if my hypothesis is correct, the original purchases should occur **before** the cancellations. By selecting the last ones, I can avoid cases whether the cancellation appears at the beginning of the dataset, where its corresponding original record might not have been captured.



In [17]:
# Slice the last 10 records
quan_nev_last_10 = quan_nev.iloc[-10:]
quan_nev_last_10 

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
540141,C581468,21314,SMALL GLASS HEART TRINKET POT,-10,2011-12-08 19:26:00,2.10,13599.0,United Kingdom
540142,C581468,22098,BOUDOIR SQUARE TISSUE BOX,-12,2011-12-08 19:26:00,0.39,13599.0,United Kingdom
540176,C581470,23084,RABBIT NIGHT LIGHT,-4,2011-12-08 19:28:00,2.08,17924.0,United Kingdom
540422,C581484,23843,"PAPER CRAFT , LITTLE BIRDIE",-80995,2011-12-09 09:27:00,2.08,16446.0,United Kingdom
540448,C581490,22178,VICTORIAN GLASS HANGING T-LIGHT,-12,2011-12-09 09:57:00,1.95,14397.0,United Kingdom
540449,C581490,23144,ZINC T-LIGHT HOLDER STARS SMALL,-11,2011-12-09 09:57:00,0.83,14397.0,United Kingdom
541541,C581499,M,Manual,-1,2011-12-09 10:28:00,224.69,15498.0,United Kingdom
541715,C581568,21258,VICTORIAN SEWING BOX LARGE,-5,2011-12-09 11:57:00,10.95,15311.0,United Kingdom
541716,C581569,84978,HANGING HEART JAR T-LIGHT HOLDER,-1,2011-12-09 11:58:00,1.25,17315.0,United Kingdom
541717,C581569,20979,36 PENCILS TUBE RED RETROSPOT,-5,2011-12-09 11:58:00,1.25,17315.0,United Kingdom


In [18]:
# Get the rows that have exactly the same 'StockCode' and 'CustomerID' as the last 10 records in the df_drop
df_drop_sc_last_10 = df_drop[df_drop['StockCode'].isin(quan_nev_last_10['StockCode']) & df_drop['CustomerID'].isin(quan_nev_last_10['CustomerID'])]
df_drop_sc_last_10

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
9197,537195,21258,VICTORIAN SEWING BOX LARGE,8,2010-12-05 13:55:00,10.95,15311.0,United Kingdom
17101,537680,22178,VICTORIAN GLASS HANGING T-LIGHT,24,2010-12-08 10:11:00,1.25,13599.0,United Kingdom
21362,538076,84978,HANGING HEART JAR T-LIGHT HOLDER,36,2010-12-09 14:15:00,1.06,15311.0,United Kingdom
21364,538076,21258,VICTORIAN SEWING BOX LARGE,8,2010-12-09 14:15:00,10.95,15311.0,United Kingdom
28606,C538650,21258,VICTORIAN SEWING BOX LARGE,-1,2010-12-13 15:05:00,10.95,15311.0,United Kingdom
...,...,...,...,...,...,...,...,...
541715,C581568,21258,VICTORIAN SEWING BOX LARGE,-5,2011-12-09 11:57:00,10.95,15311.0,United Kingdom
541716,C581569,84978,HANGING HEART JAR T-LIGHT HOLDER,-1,2011-12-09 11:58:00,1.25,17315.0,United Kingdom
541717,C581569,20979,36 PENCILS TUBE RED RETROSPOT,-5,2011-12-09 11:58:00,1.25,17315.0,United Kingdom
541744,581571,21258,VICTORIAN SEWING BOX LARGE,8,2011-12-09 12:00:00,10.95,15311.0,United Kingdom


Since there are many matches, I will examine the first 10 records for clarity.

In [19]:
# Check the first 10 records
df_drop_sc_last_10.head(10)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
9197,537195,21258,VICTORIAN SEWING BOX LARGE,8,2010-12-05 13:55:00,10.95,15311.0,United Kingdom
17101,537680,22178,VICTORIAN GLASS HANGING T-LIGHT,24,2010-12-08 10:11:00,1.25,13599.0,United Kingdom
21362,538076,84978,HANGING HEART JAR T-LIGHT HOLDER,36,2010-12-09 14:15:00,1.06,15311.0,United Kingdom
21364,538076,21258,VICTORIAN SEWING BOX LARGE,8,2010-12-09 14:15:00,10.95,15311.0,United Kingdom
28606,C538650,21258,VICTORIAN SEWING BOX LARGE,-1,2010-12-13 15:05:00,10.95,15311.0,United Kingdom
34542,539404,84978,HANGING HEART JAR T-LIGHT HOLDER,6,2010-12-17 12:24:00,1.25,17315.0,United Kingdom
42548,540005,22178,VICTORIAN GLASS HANGING T-LIGHT,15,2011-01-04 10:38:00,1.25,17315.0,United Kingdom
44161,540157,21258,VICTORIAN SEWING BOX LARGE,8,2011-01-05 11:41:00,10.95,15311.0,United Kingdom
64607,C541656,22178,VICTORIAN GLASS HANGING T-LIGHT,-1,2011-01-20 12:07:00,1.25,17315.0,United Kingdom
69108,541950,21314,SMALL GLASS HEART TRINKET POT,8,2011-01-24 12:13:00,2.10,15498.0,United Kingdom


In [20]:
# Slice the relevant transactions of 2 C invoices above
df_drop_sc_last_10_2 = df.iloc[[21364,28606,42548,64607]]
df_drop_sc_last_10_2 


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
21364,538076,21258,VICTORIAN SEWING BOX LARGE,8,2010-12-09 14:15:00,10.95,15311.0,United Kingdom
28606,C538650,21258,VICTORIAN SEWING BOX LARGE,-1,2010-12-13 15:05:00,10.95,15311.0,United Kingdom
42548,540005,22178,VICTORIAN GLASS HANGING T-LIGHT,15,2011-01-04 10:38:00,1.25,17315.0,United Kingdom
64607,C541656,22178,VICTORIAN GLASS HANGING T-LIGHT,-1,2011-01-20 12:07:00,1.25,17315.0,United Kingdom


### Interpreting Negative Values in the `Quantity` Column

From the inspected records with invoice numbers starting with `C`, it appears that some of them have corresponding orders that occurred earlier.

This suggest that these entries likely represent **cancellation transactions.** The negative values also sometimes seem to indicate **discounts**.

Therefore, I decide to retain the rows with negative values in the `Quantity` column, as they contain potentially useful information about **cancellation and discounts**.

### StockCode 'M'
While investigating negative quantities, I also noticed several rows where `StockCode` is **M** and the `Description` is **Manual**.

These transactions may represent **system- or staff-initiated corrections**, used to adjust errors or reconcile accounts.

If this assumption is correct, entries with `StockCode` == `M` should appear with both positive and negative quantities, reflecting adjustments to previous transactions rather than normal customer purchases.

In addition, `StockCode` values consisting of a single character seem to have special meanings. For example, `D` for **Discount** and `M` for **Manual**.

Therefore, I will investigate all rows where the `StockCode` is only one character long to understand whether they represent special transaction types.

In [21]:
# Select rows where'StockCode' consists of only one character
df_drop_single = df_drop[df_drop['StockCode'].str.len() == 1]

# Display the unique one-character 'StockCode' values
df_drop_single['StockCode'].unique()

array(['D', 'M'], dtype=object)

There are only two single-character 'StockCode' values in the dataset:

'D', representing **Discount**, and `M`, representing **Manual**. I have already checked **Discount**. Therefore, I will only investigate **Manual**.

In [35]:
# Check the rows where `StockCode` consists of 'M' 
df_drop_single_m = df_drop_single[df_drop_single['StockCode'] == 'M']
df_drop_single_m

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
2239,536569,M,Manual,1,2010-12-01 15:35:00,1.25,16274.0,United Kingdom
2250,536569,M,Manual,1,2010-12-01 15:35:00,18.95,16274.0,United Kingdom
6798,536981,M,Manual,2,2010-12-03 14:26:00,0.85,14723.0,United Kingdom
7976,537077,M,Manual,12,2010-12-05 11:59:00,0.42,17062.0,United Kingdom
8530,537137,M,Manual,36,2010-12-05 12:43:00,0.85,16327.0,United Kingdom
...,...,...,...,...,...,...,...,...
531432,580956,M,Manual,4,2011-12-06 14:23:00,1.25,17841.0,United Kingdom
533082,C581009,M,Manual,-1,2011-12-07 09:15:00,125.00,16971.0,United Kingdom
534308,C581145,M,Manual,-1,2011-12-07 13:48:00,9.95,17490.0,United Kingdom
538321,581405,M,Manual,3,2011-12-08 13:50:00,0.42,13521.0,United Kingdom


It seems that rows where `StockCode` == `M` represent **system- or staff-initiated corrections**, used to adjust errors or reconcile accounts, as previously suspected. 

These entries contain both positive and negative quantities, suggesting that they are not typical customer purchases. Such records could **skew the analysis** later because they do not reflect actual customer behavior. 

However, **I will retain these rows for now because excluding them would make it difficult to accurately calculate the total monetary value associated with each customer**. I will decide how to handle them after completing the data cleaning process and starting actual analysis.
 

### Check for Duplicates Records

In this step, I investigate whether the dataset contains any duplicate rows, which could distort the analysis and leas to inaccurate insights.


In [36]:
# Check the number of duplicates
df_drop.duplicated().sum()


5225

There are 5225 duplicates. Let's take a peek of some rows.

In [37]:
# Slice the duplicated rows
# Sort the rows according to each column so that the duplicated rows are shown side by side
df_drop_duprow = df_drop[df_drop.duplicated(keep=False)].sort_values(by=df_drop.columns.to_list())

df_drop_duprow.head(10)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
494,536409,21866,UNION JACK FLAG LUGGAGE TAG,1,2010-12-01 11:45:00,1.25,17908.0,United Kingdom
517,536409,21866,UNION JACK FLAG LUGGAGE TAG,1,2010-12-01 11:45:00,1.25,17908.0,United Kingdom
485,536409,22111,SCOTTIE DOG HOT WATER BOTTLE,1,2010-12-01 11:45:00,4.95,17908.0,United Kingdom
539,536409,22111,SCOTTIE DOG HOT WATER BOTTLE,1,2010-12-01 11:45:00,4.95,17908.0,United Kingdom
489,536409,22866,HAND WARMER SCOTTY DOG DESIGN,1,2010-12-01 11:45:00,2.10,17908.0,United Kingdom
527,536409,22866,HAND WARMER SCOTTY DOG DESIGN,1,2010-12-01 11:45:00,2.10,17908.0,United Kingdom
521,536409,22900,SET 2 TEA TOWELS I LOVE LONDON,1,2010-12-01 11:45:00,2.95,17908.0,United Kingdom
537,536409,22900,SET 2 TEA TOWELS I LOVE LONDON,1,2010-12-01 11:45:00,2.95,17908.0,United Kingdom
578,536412,21448,12 DAISY PEGS IN WOOD BOX,1,2010-12-01 11:49:00,1.65,17920.0,United Kingdom
598,536412,21448,12 DAISY PEGS IN WOOD BOX,1,2010-12-01 11:49:00,1.65,17920.0,United Kingdom


After inspecting the dataset, the duplicated rows appear to be pure duplicates - meaning every column value is identical. These are most likely caused by data entry or system errors, rather than genuine repeated transactions. 

The number of duplicates is **not small (5,225)**, but since the dataset contains **more than 40,000 records**, there is still plenty of data available for analysis.

Therefore, I will **remove all exact duplicate rows** to ensure data integrity and avoid overcounting in further analysis.

In [38]:
# Drop pure duplicate rows
df_dup_drop = df_drop.drop_duplicates()

# Verify that all duplicates have been removed
df_dup_drop.duplicated().sum()

0

In [39]:
# Check the info of the new DataFrame
df_dup_drop.info()

<class 'pandas.core.frame.DataFrame'>
Index: 401604 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    401604 non-null  object        
 1   StockCode    401604 non-null  object        
 2   Description  401604 non-null  object        
 3   Quantity     401604 non-null  int64         
 4   InvoiceDate  401604 non-null  datetime64[ns]
 5   UnitPrice    401604 non-null  float64       
 6   CustomerID   401604 non-null  float64       
 7   Country      401604 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 27.6+ MB


Now, all duplicated rows has been dropped.

### Investigate Inconsistency in Columns with Object Datatype



`InvoiceNo` and `StockCode` contain a mixture of numbers and characters. I will check whether these numbers are stored as *strings* to ensure data consistency and prevent potential issues in later analysis.

In [27]:
# Count datatypes in each column
inv_typ_1 = df_dup_drop['InvoiceNo'].apply(type).value_counts()
st_typ_1 = df_dup_drop['StockCode'].apply(type).value_counts()
print(inv_typ_1)
print(st_typ_1 )

InvoiceNo
<class 'int'>    392732
<class 'str'>      8872
Name: count, dtype: int64
StockCode
<class 'int'>    366179
<class 'str'>     35425
Name: count, dtype: int64


Both columns contain int. I will change all the values in both columns to string.

In [28]:
# Make a copy of df_dup_drop to avoid SettingWithCopyWarning
df_dup_drop_copy_1 = df_dup_drop.copy()

# Change all the datatype in the columns to string
df_dup_drop_copy_1['InvoiceNo'] = df_dup_drop_copy_1['InvoiceNo'].astype('str')
df_dup_drop_copy_1['StockCode'] = df_dup_drop_copy_1['StockCode'].astype('str')

# Confirm the datatype in each column is corrected
inv_typ_2 = df_dup_drop_copy_1['InvoiceNo'].apply(type).value_counts()
st_typ_2 = df_dup_drop_copy_1['StockCode'].apply(type).value_counts()

print(inv_typ_2)
print(st_typ_2)


InvoiceNo
<class 'str'>    401604
Name: count, dtype: int64
StockCode
<class 'str'>    401604
Name: count, dtype: int64


Now, all the values in `InvoiceNo` and `StockCode` columns successfully converted as strings.

Next, I will investigate `Country` column whether they contain inconsistent values like **United Kingdom** and **united kingdom**.

In [29]:
# Get the unique values in the country columns
country_unique_1 = df_dup_drop_copy_1['Country'].unique()
country_unique_1

array(['United Kingdom', 'France', 'Australia', 'Netherlands', 'Germany',
       'Norway', 'EIRE', 'Switzerland', 'Spain', 'Poland', 'Portugal',
       'Italy', 'Belgium', 'Lithuania', 'Japan', 'Iceland',
       'Channel Islands', 'Denmark', 'Cyprus', 'Sweden', 'Austria',
       'Israel', 'Finland', 'Greece', 'Singapore', 'Lebanon',
       'United Arab Emirates', 'Saudi Arabia', 'Czech Republic', 'Canada',
       'Unspecified', 'Brazil', 'USA', 'European Community', 'Bahrain',
       'Malta', 'RSA'], dtype=object)

The values in this column appear to be clean, but be certain, I will perform additional checks to verify consistency.

In [30]:
# Check the number of unique values
len(country_unique_1)

37


First, check whether there are inconsistencies with capitalization and spaces in the bigging and end.

In [31]:
# Create a new column to detect inconsistency
df_dup_drop_copy_1['Country_lower'] = df_dup_drop_copy_1['Country'].str.lower().str.strip()

print('Original unique:', df_dup_drop_copy_1['Country'].nunique())
print('Lowercase unique:', df_dup_drop_copy_1['Country_lower'].nunique())

Original unique: 37
Lowercase unique: 37


With capitalization and spaces in the begging and end, values seem to be consistence.

Next, I will check where there are typos or inconsistent spaces inside a string.

In [32]:
# Import libraries
from itertools import combinations   #   To make all unique combinations of values to compare
from rapidfuzz import fuzz

#Since there is no inconsistency with capitalization and spaces, use original 'Country' column to detect typos and inconsistent spaces
# Create a list to save the result of the check
results = []

# Make all the possible combination of unique value and calculate the similarity scores
# Threshold set to 80 to capture values that are similar but not identical
# This helps identify potential typos(e.g. 'United Kingdom vs 'United Kindom') without detecting unrelated names
# If the score is more than 80, append the combination and score in the results list

for a, b in combinations(country_unique_1, 2):
    score = fuzz.ratio(a, b)
    if score >= 80:
        results.append((a, b, score))

# Print the result
print(results)

[('Australia', 'Austria', 87.5)]


`Australia` and `Austria` are different country and not a typo and only this pair is detected, which means there aren't inconsistent spaces and typos. Therefore, `Country` column is clean and consistent.

In [33]:
# Drop the column used for validation and check the information of the cleaned DataFrame
df_dup_drop_copy_1 = df_dup_drop_copy_1.drop(columns='Country_lower')
df_dup_drop_copy_1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 401604 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    401604 non-null  object        
 1   StockCode    401604 non-null  object        
 2   Description  401604 non-null  object        
 3   Quantity     401604 non-null  int64         
 4   InvoiceDate  401604 non-null  datetime64[ns]
 5   UnitPrice    401604 non-null  float64       
 6   CustomerID   401604 non-null  float64       
 7   Country      401604 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 27.6+ MB


### RFM Analysis

**RFM** stands for **Recency**, **Frequency** and **Monetary**. These are three key metrics used in marketing and customer analytics to describe customer behavior.

1. **Recency**
How recently customer made a purchase - The number of days since the customer's last purchase(relative to a reference date, e.g. the last date in your dataset)

2. **Frequency**
How of often a customer makes purchases - The total number of invoices(transactions) by the customers

3. **Monetary**
How much money the customer spent - The total monetary value of purchase

### Problems of Rows with minus quantities and 'StockCODE' == 'M'
As I investigated above, there are rows with negative quantities that are **discounts** and **cancellation**. These rows could skew the outcomes of RFM analysis. Therefore I will conduct the analysis in the following steps.

1. **Separate the DataFrame**
Separate the DataFrame into 3, which are **rows with discounts**, **rows with cancellation** and **rows with positive quantity**, which are pure purchase histories.

2. **Compute RFM, but with M as adjusted Monetary**

3. 



In [34]:
# Create RFM Columns
df = df_dup_drop_copy_1[df_dup_drop_copy_1['StockCode'].str.len() == 1]
df

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
141,C536379,D,Discount,-1,2010-12-01 09:41:00,27.50,14527.0,United Kingdom
2239,536569,M,Manual,1,2010-12-01 15:35:00,1.25,16274.0,United Kingdom
2250,536569,M,Manual,1,2010-12-01 15:35:00,18.95,16274.0,United Kingdom
6798,536981,M,Manual,2,2010-12-03 14:26:00,0.85,14723.0,United Kingdom
7976,537077,M,Manual,12,2010-12-05 11:59:00,0.42,17062.0,United Kingdom
...,...,...,...,...,...,...,...,...
531432,580956,M,Manual,4,2011-12-06 14:23:00,1.25,17841.0,United Kingdom
533082,C581009,M,Manual,-1,2011-12-07 09:15:00,125.00,16971.0,United Kingdom
534308,C581145,M,Manual,-1,2011-12-07 13:48:00,9.95,17490.0,United Kingdom
538321,581405,M,Manual,3,2011-12-08 13:50:00,0.42,13521.0,United Kingdom
